# Character Impact Project (Part 8) - Facial Recognition on Video
## Deep Learning Facial Recognition
### Alden Chico

---

## Premise

From the last section of the blog series, we attained a 0.9975 F1 classification score on our 500+ image dataset of characters from the first season of <i>The Office</i>. We produced these amazing results by segmenting faces from images using a Multi-Task Convolutional Neural Network, embedding these face images using the deep learning based FaceNet, and training a Support Vector Machine to classify vectors to their corresponding character's labels. Now that we have this high-performing and robust approach to facial recognition, this blog post will focus on integrating the lessons we learned from the past two posts about facial recognition to actual video data from <i>The Office</i>.

<figure>
<img src="http://netflinda.com/wp-content/uploads/2018/10/Untitled-design.png" alt="Michael Scott Rocking Out" class="center" style=width:449px;height:300px;>
<figcaption><center><b>This is the end y'all!!!</b></center></figcaption>
</figure> 

The first thing we need to do is create a method that can import a video file from the project directory and perform facial recognition on every frame of that video file. We will accomplish this using OpenCV. In order to measure the amount of screen time each character has in the video, we will log every frame that the facial classifier recognized the character's face in the frame and record the segmented face to memory. By using the framerate of the video, we can  measure how many seconds each character spent on screen by dividing the number of frames that the classifier found the character by the frame rate of the video.

After gathering saved segmented face images from whole episodes of <i>The Office</i> Season One, we will then create a method that can organize character screen time into a dictionary. With a dictionary that holds the amount of screen time each character has in the video, we can use that information to generate a mock IMDb website page that shows how much time each character spends on screen. <b>We will ultimately add these character screen time measurements to a mock version of the IMDb website</b>. This is the final stretch of our project, so without further adieu, let's begin.

<figure>
<img src="reference/8-Facial_Recognition_Video/test/output/IMDb/IMDb-Screenshot.png" alt="IMDb Screenshot" class="center"style="width:523px;height:500px;">
<figcaption><center><b>End Result.</b></center></figcaption>
</figure> 

---

## Facial Recognition on Video

Our first goal for this blog post is implementing facial recognition on video data. To do this, we'll start by importing all the methods that we wrote from the last blog post using ```import_ipynb```.

In [1]:
# Importing functions from the last part of the project's ipynb file as a library for this part
import import_ipynb
from Face_Recognition_Module_2 import *

importing Jupyter notebook from Face_Recognition_Module_2.ipynb
importing Jupyter notebook from Facial_Recognition_Module.ipynb


Using TensorFlow backend.


Before we can start using facial recognition on video data, we need to first generate an SVM model that's trained on the FaceNet vectors of characters from the show.

In the code below, we train the SVM on 1000 randomly generated augmented training images per character in our dataset. Since this process takes a long time to finish, I ran through the lines of code that accomplish this once and commented them out. Image augmentation takes place in the ```augment_training_images``` method commented in Step 2. In Step 3, the ```embed_augment_face_dict``` object is a defaultdict that stores all the FaceNet vectors into lists that are organized by the character's label number. And finally, in Step 4, the  ```train_SVM``` and ```dump``` methods generate an SVM model that's trained on the FaceNet training vectors and save the trained model into an svm.txt pickle file, respectively. Having this pickle file is useful for storing the trained machine learning model for use later in the project.

So now that we have a trained SVM saved in the project directory, the only preparation now that's really important is loading the FaceNet model using the ```load_model``` method from keras, loading the MTCNN model from the ```mtcnn```library, and loading the trained SVM using the ```load``` method from the pickle library.

In [2]:
from pickle import dump, load

In [3]:
# Step 1: Organize training images into a default_dict
character_dict, original_list, label_list = prepare_data('reference/8-Facial_Recognition_Video/*/*.png', -2)
img_dict, _ = prepare_image_dictionaries(original_list, label_list, [])

############################################################################################################

# Step 2: Augment the training images
'''
Augment the training images to 1000 sample training images
This has already been done and stored in project directory
'''
project_path = 'reference/8-Facial_Recognition_Video'
#_ = augment_training_images(character_dict, project_path, img_dict, num_samples=1000, random_state=42)

############################################################################################################

# Step 3: Embed the training images using the FaceNet classifier
'''
embed_augment_face_dict is used to train SVM classifier
Pre-trained SVM classifier stored in project directory as svm.txt
'''
feature_extractor = load_model('facenet_keras.h5')
#embed_augment_face_dict = embed_training_images(project_path, feature_extractor)

############################################################################################################

# Step 4: Train SVM using training vector list and label list
'''
Created a pickle txt file that stores the attributes of SVM trained classifier
This has already been done and the file is stored in project directory
'''

#model = train_SVM(embed_augment_face_dict)
#dump(model, open('reference/8-Facial_Recognition_Video/svm.txt', 'wb'))






Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/media/qiujing/e5439522-63c4-4b7c-a968-fefee6a3d960/omead/anaconda3/envs/face_project/lib/python3.7/site-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


'\nCreated a pickle txt file that stores the attributes of SVM trained classifier\nThis has already been done and the file is stored in project directory\n'

---

With all our models loaded, we can start working on facial recognition from video data. The following method ```facial_recognition_on_video``` is pretty long, but in essence, the method takes the input video and reads it frame by frame. After reading the frame, the method segments faces from the image using MTCNN. If faces are found, the method goes through each individual face and embeds it into a FaceNet vector. The method then predicts the label for the vector using the trained SVM. If the highest confidence for prediction is greater than 0.85 (a threshold gathered using trial and error), a green rectangle and predicted name is drawn on the frame. The segmented face image is saved into a prediction directory for the character. This process repeats until every frame is written to the output video file.

In [4]:
import cv2
from mtcnn.mtcnn import MTCNN

In [5]:
''' Takes input video from input_vid_path and writes output video with predictions to output_vid_path'''

def facial_recognition_on_video(input_vid_path, output_vid_path, frame_size, fps, 
                                character_dict, project_path):

    # Load the reference video into OpenCV
    cap = cv2.VideoCapture(input_vid_path)

    # Prepare the video writer
    fourcc = cv2.VideoWriter_fourcc('M', 'P', '4', 'V')
    out = cv2.VideoWriter(output_vid_path, fourcc=fourcc, fps=fps, frameSize=frame_size)

    # Load MTCNN and FaceNet pre-trained classifiers
    detector = MTCNN()
    feature_extractor = load_model('facenet_keras.h5')
    green = (0, 255, 0)

    # Load the pre-trained SVM classifier into the project
    pickle_file = open('reference/8-Facial_Recognition_Video/svm.txt', 'rb')
    model = load(pickle_file)

    # Make prediction directories in test directory labelled with the name of the input video file
    vid_name = input_vid_path.split('/')[-1].split('.')[0]
    path = project_path + '/test/output/' + vid_name + '_predictions'
    _ = make_prediction_directories(character_dict, path)
    while(cap.isOpened()):

        # Read a single frame from the video
        ret, frame = cap.read()
        if ret == True:

            # Apply MTCNN to detect faces in the frame
            frame_copy = frame.copy()
            faces = detector.detect_faces(frame_copy)

            # Draw rectangle along with predicted faces if the probability is greater than 0.75
            for face in faces:

                # Segment the face from the image
                (x, y, w, h) = face['box']
                face_img = frame_copy[y:y+h, x:x+w]

                # Reject the face if the box has a 0 width/height dimension
                if(face_img.shape[0] == 0 or face_img.shape[1] == 0):
                    continue

                # Create a FaceNet embedding for the face
                embedding = embed_face(face_img, feature_extractor)
                embedding = np.expand_dims(embedding, axis=0)

                # Predict the character's name to the face embedding using the pre-trained SVM model
                prediction = model.predict_proba(embedding)
                max_pred_prob = max(prediction[0])
                pred_label = np.argmax(prediction)

                # Draw rectangle around the character's face if the maximum predicted probability is greater than 0.85
                if max_pred_prob >= 0.85:
                    character = character_dict[pred_label]
                    cv2.rectangle(frame_copy, (x, y), (x+w, y+h), green, 2)
                    cv2.putText(frame_copy, character, (x, y-15), cv2.FONT_HERSHEY_PLAIN, 
                                fontScale=1.5, color=green, thickness=2)

                    # Write the face into the character's prediction directory
                    i = 1
                    prediction_path = path + '/' + character + '/prediction'
                    while os.path.exists("{}/{}.png".format(prediction_path, i)):
                        i += 1
                    img_path = '{}/{}.png'.format(prediction_path, i)
                    _ = cv2.imwrite(img_path, face_img)

            # Write the frame with character predictions out using the video writer
            out.write(frame_copy)
        else:
            break


    # Release the capture and video writer
    cap.release()
    out.release()
    cv2.destroyAllWindows()

---

Let's see ```facial_recognition_on_video``` in action. In our project directory, we have a highlight clip from the second episode of the show, "Diversity Day". We want to save the output in our <b>test/output</b> directory as <b>test_video_output.mp4</b>. We provide the function with the resolution and frame rate as well as ```character_dict``` and ```project_path``` that we defined before. After all that's done, we create a ```clips_array``` using the ```moviepy.editor``` library to show a side-by-side comparison of the non-edited and edited videos as <b>comparison.mp4</b>.

In [6]:
input_vid_path = 'reference/8-Facial_Recognition_Video/test/test_video.mp4'
output_vid_path = 'reference/8-Facial_Recognition_Video/test/output/test_video_output.mp4'
frame_size = (636, 360)
fps = 30
_ = facial_recognition_on_video(input_vid_path, output_vid_path, 
                                frame_size, fps, character_dict, project_path)

In [7]:
from moviepy.editor import VideoFileClip, clips_array

In [8]:
# Convert the face detected videos into MoviePy VideoFileClip objects
clip1 = VideoFileClip('reference/8-Facial_Recognition_Video/test/test_video.mp4').margin(10)
clip2 = VideoFileClip('reference/8-Facial_Recognition_Video/test/output/test_video_output.mp4').margin(10)

# Edit the videos side-by-side
final_clip = clips_array([[clip1, clip2]])

# Save the edited video
final_clip.resize(width=640).write_videofile('reference/8-Facial_Recognition_Video/test/comparison.mp4', audio=False,
                                            verbose=False);

t:   0%|          | 11/4111 [00:00<00:37, 107.91it/s, now=None]

Moviepy - Building video reference/8-Facial_Recognition_Video/test/comparison.mp4.
Moviepy - Writing video reference/8-Facial_Recognition_Video/test/comparison.mp4



Moviepy - Done !
Moviepy - video ready reference/8-Facial_Recognition_Video/test/comparison.mp4


In [9]:
from IPython.display import HTML

In [10]:
%%HTML
<video width="640" height="360" controls>
  <source src="reference/8-Facial_Recognition_Video/test/comparison.mp4" type="video/mp4">
</video>

---

## Timing Characters on Video

Now that our facial recognition pipeline works on video data, it's time to measure character screen time. Since we saved the segmented face images from every frame into the character's prediction folder, we can count the number of frames for that character. After retrieving a character frame count, we can divide that number by the frame rate of the video to get the number of seconds the character had on screen. Once this is done, we can format the screen time into an "mm:ss" format and add it to a dictionary. The dictionary keys are organized by the names of the character. 

In [11]:
from collections import defaultdict
from glob import glob

In [12]:
def time_characters(project_path, input_vid_path, fps):
    
    vid_name = input_vid_path.split('/')[-1].split('.')[0]
    path = project_path + '/test/output/' + vid_name + '_predictions'
    sec_count = {}
    for c_label, character in character_dict.items():

        # Find how many frames were found for each character in the video and divide by FPS for time in seconds
        glob_pattern = path + '/' + character + '/prediction/*'
        img_path_list = glob(glob_pattern)
        sec_count[character] = len(img_path_list) / fps
        minutes = int(sec_count[character] / 60)
        seconds = int(sec_count[character] % 60)
        if seconds < 10:
            sec_count[character] = '{}:0{}'.format(minutes, seconds)
        else:
            sec_count[character] = '{}:{}'.format(minutes, seconds)
    return sec_count

In [13]:
character = 'michael'
fps = 30
input_vid_path = 'reference/8-Facial_Recognition_Video/test/test_video.mp4'
project_path = 'reference/8-Facial_Recognition_Video'
sec_count = time_characters(project_path, input_vid_path, fps)
print('character: {}'.format(character))
print('time on screen: {} seconds'.format(sec_count[character]))

character: michael
time on screen: 0:37 seconds


---

With all that work behind us, it's finally time to run ```facial_recognition_on_video``` on the first season of <i>The Office</i>. It takes quite a while to run the following code block since there are hundreds of thousands of frames to process for the 6 episode dataset that we have. I ran the code over night and it was done processing when I woke up around 7 hours later. I can verify that the facial classifier ran properly on all the episodes from the first season, but I won't be posting the whole episodes on my GitHub. Instead, if you're interested in the results, there are prediction image folders that contain all the segmented faces from running the classifier on the first season. With the segmented face images saved into the prediction directories, we can now add screen time measurements to a pandas DataFrame.

In [14]:
# Create a list of episode names from the test project directory
ep_path = 'reference/8-Facial_Recognition_Video/test/The_Office_*'
ep_path_list = glob(ep_path)
for ep in ep_path_list:
    
    # Input video path is the episode path from ep_path_list
    input_vid_path = ep
    input_vid_name = input_vid_path.split('/')[-1].split('.')[0]
    
    # Construct output video path using name of episode + _output.mp4
    output_vid_name =  input_vid_name + '_output.mp4'
    dir_path_list = input_vid_path.split('/')[:-1]
    output_vid_path = ''
    for p in dir_path_list:
        output_vid_path += p + '/'
    output_vid_path = output_vid_path + 'output/' + output_vid_name
    
    # Perform facial recognition on the episode video file
    frame_size = (640, 360)
    fps = 24
    _ = facial_recognition_on_video(input_vid_path, output_vid_path, frame_size, fps, character_dict, project_path)

---

## Recording Screen Time Measurements to a pandas DataFrame

The following pandas DataFrame is from the third part of our blog series where we performed exploratory data analysis on information gathered from the IMDb website. Our goal here is to add a row for each episode where we can insert character screen time measurements. We can do this by splitting the DataFrame into two halves, adding the screen time row to the upper half, and concatenating the two halves together again. We repeat this process until all episodes have a screen time row. The results of doing this can be seen below.

In [15]:
import pandas as pd

In [16]:
full_episode_df = pd.read_csv('reference/8-Facial_Recognition_Video/full_episode_df.csv')
full_episode_df

,Episode,Unnamed: 1,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Pilot,Cast,Steve Carell,Rainn Wilson,John Krasinski,Jenna Fischer,B.J. Novak,Melora Hardin,David Denman,Leslie David Baker,Brian Baumgartner,Angela Kinsey,Henriette Mantel,Mike McCaul,Oscar Nuñez,Phyllis Smith
1,Pilot,Characters,Michael Scott,Dwight Schrute,Jim Halpert,Pam Beesly,Ryan Howard,Jan Levinson-Gould,Roy Anderson,Stanley Hudson,Kevin Malone,Angela Martin,Office Worker,Office Worker,Oscar Martinez,Phyllis Lapin
2,Diversity Day,Cast,Steve Carell,Rainn Wilson,John Krasinski,Jenna Fischer,B.J. Novak,Larry Wilmore,Leslie David Baker,Brian Baumgartner,Kate Flannery,Mindy Kaling,Paul Lieberstein,Angela Kinsey,Oscar Nuñez,Phyllis Smith
3,Diversity Day,Characters,Michael Scott,Dwight Schrute,Jim Halpert,Pam Beesly,Ryan Howard,Mr. Brown,Stanley Hudson,Kevin Malone,Meredith Palmer,Kelly Kapoor,Toby Flenderson,Angela Martin,Oscar Martinez,Phyllis Lapin
4,Health Care,Cast,Steve Carell,Rainn Wilson,John Krasinski,Jenna Fischer,B.J. Novak,Melora Hardin,Leslie David Baker,Brian Baumgartner,Kate Flannery,Charlie Hartsock,Mindy Kaling,Angela Kinsey,Paul Lieberstein,Oscar Nuñez
5,Health Care,Characters,Michael Scott,Dwight Schrute,Jim Halpert,Pam Beesly,Ryan Howard,Jan Levinson-Gould,Stanley Hudson,Kevin Malone,Meredith Palmer,Travel Agent,Kelly Kapoor,Angela Martin,Toby Flenderson,Oscar Martinez
6,The Alliance,Cast,Steve Carell,Rainn Wilson,John Krasinski,Jenna Fischer,B.J. Novak,David Denman,Craig Robinson,Leslie David Baker,Brian Baumgartner,Kate Flannery,Paul Lieberstein,Angela Kinsey,Oscar Nuñez,Phyllis Smith
7,The Alliance,Characters,Michael Scott,Dwight Schrute,Jim Halpert,Pam Beesly,Ryan Howard,Roy Anderson,Darryl Philbin,Stanley Hudson,Kevin Malone,Meredith Palmer,Toby Flenderson,Angela Martin,Oscar Martinez,Phyllis Lapin
8,Basketball,Cast,Steve Carell,Rainn Wilson,John Krasinski,Jenna Fischer,B.J. Novak,David Denman,Craig Robinson,Patrice O'Neal,Leslie David Baker,Brian Baumgartner,Matt DeCaro,Kate Flannery,Angela Kinsey,Oscar Nuñez
9,Basketball,Characters,Michael Scott,Dwight Schrute,Jim Halpert,Pam Beesly,Ryan Howard,Roy Anderson,Darryl Philbin,Lonny,Stanley Hudson,Kevin Malone,Jerry,Meredith Palmer,Angela Martin,Oscar Martinez


In [17]:
'''Add Screen Time row for every episode in the DataFrame'''

full_episode_df_copy = full_episode_df.copy()

end = 3
num_episodes = 6
# Split the DataFrame into two halves
a = full_episode_df_copy.iloc[:end,:].copy()
b = full_episode_df_copy.iloc[end-1:,:].copy()
c = pd.DataFrame()
for i in range(num_episodes):
    
    # Append last Screen Time row on the last episode
    if i == num_episodes-1:
        b = a.iloc[-1:, :].copy()
        b.iloc[-1,1] = 'Screen Time'
        b.iloc[-1,2:] = '0:00'
        c = pd.concat([a, b])
        c = c.reset_index(drop=True)
        break
    
    # Insert a Screen Time row for every episode in the season    
    a.iloc[-1,0] = a.iloc[-2,0]
    a.iloc[-1,1] = 'Screen Time'
    a.iloc[-1,2:] = '0:00'
    c = pd.concat([a, b])
    c = c.reset_index(drop=True)
    
    # Update the DataFrame splits
    end += 3
    a = c.iloc[:end,:].copy()
    b = c.iloc[end-1:,:].copy()

# Write out the edited DataFrame to a CSV file
c.to_csv('reference/8-Facial_Recognition_Video/full_episode_df_edit.csv')
edited_df = pd.read_csv('reference/8-Facial_Recognition_Video/full_episode_df_edit.csv',index_col=(1,2))
edited_df = edited_df.drop(columns=['Unnamed: 0'])
edited_df

1               2               3  \
Episode       Unnamed: 1                                                   
Pilot         Cast          Steve Carell    Rainn Wilson  John Krasinski   
              Characters   Michael Scott  Dwight Schrute     Jim Halpert   
              Screen Time           0:00            0:00            0:00   
Diversity Day Cast          Steve Carell    Rainn Wilson  John Krasinski   
              Characters   Michael Scott  Dwight Schrute     Jim Halpert   
              Screen Time           0:00            0:00            0:00   
Health Care   Cast          Steve Carell    Rainn Wilson  John Krasinski   
              Characters   Michael Scott  Dwight Schrute     Jim Halpert   
              Screen Time           0:00            0:00            0:00   
The Alliance  Cast          Steve Carell    Rainn Wilson  John Krasinski   
              Characters   Michael Scott  Dwight Schrute     Jim Halpert   
              Screen Time           0:00            0:00            0:00   
Basketball    Cast          Steve Carell    Rainn Wilson  John Krasinski   
              Characters   Michael Scott  Dwight Schrute     Jim Halpert   
              Screen Time           0:00            0:00            0:00   
Hot Girl      Cast          Steve Carell    Rainn Wilson  John Krasinski   
              Characters   Michael Scott  Dwight Schrute     Jim Halpert   
              Screen Time           0:00            0:00            0:00   

                                       4            5                   6  \
Episode       Unnamed: 1                                                    
Pilot         Cast         Jenna Fischer   B.J. Novak       Melora Hardin   
              Characters      Pam Beesly  Ryan Howard  Jan Levinson-Gould   
              Screen Time           0:00         0:00                0:00   
Diversity Day Cast         Jenna Fischer   B.J. Novak       Larry Wilmore   
              Characters      Pam Beesly  Ryan Howard           Mr. Brown   
              Screen Time           0:00         0:00                0:00   
Health Care   Cast         Jenna Fischer   B.J. Novak       Melora Hardin   
              Characters      Pam Beesly  Ryan Howard  Jan Levinson-Gould   
              Screen Time           0:00         0:00                0:00   
The Alliance  Cast         Jenna Fischer   B.J. Novak        David Denman   
              Characters      Pam Beesly  Ryan Howard        Roy Anderson   
              Screen Time           0:00         0:00                0:00   
Basketball    Cast         Jenna Fischer   B.J. Novak        David Denman   
              Characters      Pam Beesly  Ryan Howard        Roy Anderson   
              Screen Time           0:00         0:00                0:00   
Hot Girl      Cast         Jenna Fischer   B.J. Novak           Amy Adams   
              Characters      Pam Beesly  Ryan Howard                Katy   
              Screen Time           0:00         0:00                0:00   

                                            7                   8  \
Episode       Unnamed: 1                                            
Pilot         Cast               David Denman  Leslie David Baker   
              Characters         Roy Anderson      Stanley Hudson   
              Screen Time                0:00                0:00   
Diversity Day Cast         Leslie David Baker   Brian Baumgartner   
              Characters       Stanley Hudson        Kevin Malone   
              Screen Time                0:00                0:00   
Health Care   Cast         Leslie David Baker   Brian Baumgartner   
              Characters       Stanley Hudson        Kevin Malone   
              Screen Time                0:00                0:00   
The Alliance  Cast             Craig Robinson  Leslie David Baker   
              Characters       Darryl Philbin      Stanley Hudson   
              Screen Time                0:00                0:00   
Basketball 

---

With the added screen time rows, we can add the actual screen time measurements for each episode of the first season. Using the ```time_characters``` method we made before, we will create a dictionary that has all the character screen times populated in the dictionary. From there, we can create a super dictionary called ```episode_sec_count``` that stores all the episode screen time dictionaries. The key values for ```episode_sec_count``` are the names of the episodes from the first season of the show. With ```episode_sec_count```, we can populate the character screen time values to our show's DataFrame.

In [18]:
from glob import glob

In [19]:
'''
A dictionary with video file names as the keys and episode names as the values
'''
episode_titles = {
    'The_Office_S01E01':'Pilot',
    'The_Office_S01E02':'Diversity Day',
    'The_Office_S01E03':'Health Care',
    'The_Office_S01E04':'The Alliance',
    'The_Office_S01E05':'Basketball',
    'The_Office_S01E06':'Hot Girl'
}

In [20]:
'''Create a dictionary episode_sec_count that contains all the episodes' character screen times referenced by their 
    episode title'''

fps = 24
project_path = 'reference/8-Facial_Recognition_Video'
input_vid_path_pattern = 'reference/8-Facial_Recognition_Video/test/The_Office_*'
input_vid_path_list = glob(input_vid_path_pattern)

episode_sec_count = {}
for input_vid_path in input_vid_path_list:
    vid_name = input_vid_path.split('/')[-1].split('.')[0]
    episode_title = episode_titles[vid_name]
    sec_count = time_characters(project_path, input_vid_path, fps)
    episode_sec_count[episode_title] = sec_count

In [21]:
'''
A dictionary that will help convert the short names for the characters from the show 
with their full names
'''

full_character_name_dict = {
    'angela':'Angela Martin',
    'ryan':'Ryan Howard',
    'kevin':'Kevin Malone',
    'pam':'Pam Beesly',
    'jim':'Jim Halpert',
    'stanley':'Stanley Hudson',
    'oscar':'Oscar Martinez',
    'dwight':'Dwight Schrute',
    'michael':'Michael Scott',
    'meredith':'Meredith Palmer',
    'roy':'Roy Anderson',
    'toby':'Toby Flenderson',
    'jan':'Jan Levinson-Gould',
    'phyllis':'Phyllis Lapin',
    'daryl':'Darryl Philbin',
    'kelly':'Kelly Kapoor',
    'katy':'Katy',
    'lonny':'Lonny'
}

In [22]:
'''Edit the DataFrame with measured character screen times'''

edited_df_copy = edited_df.copy()
for ep_title, ep_screen_time in episode_sec_count.items():
    # Go through every row of the DataFrame
    for idx, row in edited_df.iterrows():
        # Select the screen time row for the appropriate episode
        if idx[0] == ep_title and idx[1] == 'Screen Time':
            # Go through each character in the episode's screen time dictionary
            for character, t in ep_screen_time.items():
                full_name = full_character_name_dict[character]
                # Go through every column in the screen time row to find the character's screen time cell
                for col, _ in edited_df_copy.iteritems():
                    # Edit the appropriate cell with the character's screen time
                    if edited_df_copy.loc[(ep_title, 'Characters'), col] == full_name:
                        edited_df_copy.loc[(ep_title, 'Screen Time'), col] = t
edited_df_copy.to_csv('reference/8-Facial_Recognition_Video/full_episode_df_screen_time.csv')

In [23]:
edited_df_copy = pd.read_csv('reference/8-Facial_Recognition_Video/full_episode_df_screen_time.csv', index_col=[0,1])
edited_df_copy

1               2               3  \
Episode       Unnamed: 1                                                   
Pilot         Cast          Steve Carell    Rainn Wilson  John Krasinski   
              Characters   Michael Scott  Dwight Schrute     Jim Halpert   
              Screen Time           8:06            2:58            2:00   
Diversity Day Cast          Steve Carell    Rainn Wilson  John Krasinski   
              Characters   Michael Scott  Dwight Schrute     Jim Halpert   
              Screen Time           6:43            1:31            2:03   
Health Care   Cast          Steve Carell    Rainn Wilson  John Krasinski   
              Characters   Michael Scott  Dwight Schrute     Jim Halpert   
              Screen Time           4:24            4:05            1:10   
The Alliance  Cast          Steve Carell    Rainn Wilson  John Krasinski   
              Characters   Michael Scott  Dwight Schrute     Jim Halpert   
              Screen Time           5:13            2:37            1:10   
Basketball    Cast          Steve Carell    Rainn Wilson  John Krasinski   
              Characters   Michael Scott  Dwight Schrute     Jim Halpert   
              Screen Time           4:17            1:02            1:25   
Hot Girl      Cast          Steve Carell    Rainn Wilson  John Krasinski   
              Characters   Michael Scott  Dwight Schrute     Jim Halpert   
              Screen Time           6:17            2:37            1:03   

                                       4            5                   6  \
Episode       Unnamed: 1                                                    
Pilot         Cast         Jenna Fischer   B.J. Novak       Melora Hardin   
              Characters      Pam Beesly  Ryan Howard  Jan Levinson-Gould   
              Screen Time           3:12         0:37                0:49   
Diversity Day Cast         Jenna Fischer   B.J. Novak       Larry Wilmore   
              Characters      Pam Beesly  Ryan Howard           Mr. Brown   
              Screen Time           0:42         0:25                0:00   
Health Care   Cast         Jenna Fischer   B.J. Novak       Melora Hardin   
              Characters      Pam Beesly  Ryan Howard  Jan Levinson-Gould   
              Screen Time           1:50         0:09                0:18   
The Alliance  Cast         Jenna Fischer   B.J. Novak        David Denman   
              Characters      Pam Beesly  Ryan Howard        Roy Anderson   
              Screen Time           1:32         0:16                0:01   
Basketball    Cast         Jenna Fischer   B.J. Novak        David Denman   
              Characters      Pam Beesly  Ryan Howard        Roy Anderson   
              Screen Time           1:44         1:07                0:42   
Hot Girl      Cast         Jenna Fischer   B.J. Novak           Amy Adams   
              Characters      Pam Beesly  Ryan Howard                Katy   
              Screen Time           2:16         0:20                1:36   

                                            7                   8  \
Episode       Unnamed: 1                                            
Pilot         Cast               David Denman  Leslie David Baker   
              Characters         Roy Anderson      Stanley Hudson   
              Screen Time                0:44                0:03   
Diversity Day Cast         Leslie David Baker   Brian Baumgartner   
              Characters       Stanley Hudson        Kevin Malone   
              Screen Time                0:41                1:19   
Health Care   Cast         Leslie David Baker   Brian Baumgartner   
              Characters       Stanley Hudson        Kevin Malone   
              Screen Time                0:21                1:08   
The Alliance  Cast             Craig Robinson  Leslie David Baker   
              Characters       Darryl Philbin      Stanley Hudson   
              Screen Time                0:01                0:15   
Basketball 

---

## Editing IMDb's Website to Show Screen Time Measurements

To end this project off, we will dive into IMDb's HTML and edit the HTML code to show character screen time. To do this, we can use BeautifulSoup. What we do in the code cells below is parse through the IMDb cast_list table attribute and go through each individual cast member for a single episode. The episodes are referenced in IMDb's website using a <b>tconst</b> primary key that we retrieved in a previous part of our project. 

Using tconst, we access the IMDb webpage's HTML code using the ```requests``` library. We insert a new attribute to the <b>cast_list table</b> called <b>screen_time</b>that will store all the screen time information for the characters from the show. From there, we populate the attribute with the appropriate character's screen time and repeat the process for all the episodes that we classified on. 

The last thing we do is save the edited HTML code to a .txt file and edit the IMDb HTML code using the Inspect tool on Google Chrome web browser. The video below shows the end result of this entire process.

In [24]:
from bs4 import BeautifulSoup
import requests

In [25]:
imdb_df = pd.read_csv('reference/8-Facial_Recognition_Video/the_office_imdb.csv', index_col=None)
imdb_df

,Unnamed: 0,tconst_series,tconst_episode,primaryTitle_series,primaryTitle_episode,seasonNumber,episodeNumber,averageRating,cast,characters
0,0,tt0386676,tt0664521,The Office,Pilot,1,1,7.6,"['Steve Carell', 'Rainn Wilson', 'John Krasins...","['Michael Scott', 'Dwight Schrute', 'Jim Halpe..."
1,1,tt0386676,tt0664514,The Office,Diversity Day,1,2,8.3,"['Steve Carell', 'Rainn Wilson', 'John Krasins...","['Michael Scott', 'Dwight Schrute', 'Jim Halpe..."
2,2,tt0386676,tt0664517,The Office,Health Care,1,3,7.9,"['Steve Carell', 'Rainn Wilson', 'John Krasins...","['Michael Scott', 'Dwight Schrute', 'Jim Halpe..."
3,3,tt0386676,tt0664523,The Office,The Alliance,1,4,8.1,"['Steve Carell', 'Rainn Wilson', 'John Krasins...","['Michael Scott', 'Dwight Schrute', 'Jim Halpe..."
4,4,tt0386676,tt0664510,The Office,Basketball,1,5,8.4,"['Steve Carell', 'Rainn Wilson', 'John Krasins...","['Michael Scott', 'Dwight Schrute', 'Jim Halpe..."
5,5,tt0386676,tt0664518,The Office,Hot Girl,1,6,7.8,"['Steve Carell', 'Rainn Wilson', 'John Krasins...","['Michael Scott', 'Dwight Schrute', 'Jim Halpe..."


In [28]:
imdb_df = imdb_df.astype('object')

table_dic = {}
for (ep_num, ep), (_, ep_title) in zip(imdb_df.tconst_episode.iteritems(), imdb_df.primaryTitle_episode.iteritems()):
    
    #Construct the IMDb URL and extract HTML data
    x = ep_num + 1
    url = f'https://www.imdb.com/title/{ep}/?ref_=ttep_ep{x}' 
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    
    # Construct the first list of characters from HTML data with 'odd' class tags
    characters_odd = soup.find_all('tr', class_='odd')
    for character in characters_odd:
        
        # Importing the HTML using beautifulsoup changes image properties, so edit images to display correctly
        img_tag = character.find("td", class_='primary_photo')
        img_tag = img_tag.find("a").contents[0]
        img_tag['class'] = 'loadlate'
        # BeautifulSoup had a hard time finding 'loadlate' attribute, so had to use this workaround
        for key, val in img_tag.attrs.items():
            if key == 'loadlate':
                img_tag['src']= img_tag[key]
        
        # Add the character's screen time to HTML
        new_tag = soup.new_tag("td", **{'class':'screen_time'})
        new_tag.string = "0:00"
        c_name = character.find('td', class_='character').text.replace('\n','').strip()
        for name, full_name in full_character_name_dict.items():
            if full_name == c_name:
                new_tag.string = episode_sec_count[ep_title][name]
                break
        character.append(new_tag)
    
    # Construct the first list of characters from HTML data with 'odd' class tags
    characters_even = soup.find_all('tr', class_='even')
    for character in characters_even:
        
        # Importing the HTML using beautifulsoup changes image properties, so edit images to display correctly
        img_tag = character.find("td", class_='primary_photo')
        img_tag = img_tag.find("a").contents[0]
        img_tag['class'] = 'loadlate'
        
        # BeautifulSoup had a hard time finding 'loadlate' attribute, so had to use this workaround
        for key, val in img_tag.attrs.items():
            if key == 'loadlate':
                img_tag['src']= img_tag[key]
        
        # Add the character's screen time to HTML
        new_tag = soup.new_tag("td", **{'class':'screen_time'})
        new_tag.string = "0:00"
        c_name = character.find('td', class_='character').text.replace('\n','').strip()
        for name, full_name in full_character_name_dict.items():
            if full_name == c_name:
                new_tag.string = episode_sec_count[ep_title][name]
                break
        character.append(new_tag)
        
    imdb_table = soup.find('table', class_='cast_list')
    table_dic[ep_title] = imdb_table

In [30]:
'''
Save the IMDb files to an output IMDb project directory
'''

for ep_name, content in table_dic.items():
    txt_file = 'reference/8-Facial_Recognition_Video/test/output/IMDb/' + ep_name + '_IMDb.txt' 
    with open(txt_file, 'w', encoding='utf-8') as f_out:
        f_out.write(content.prettify())

In [31]:
%%HTML
<video width="640" height="360" controls>
  <source src="reference/8-Facial_Recognition_Video/test/output/IMDb/IMDb_Demonstration.mp4" 
    type="video/mp4">
</video>

And there you have it. We have successfully added the character screen times to IMDb's website! 

We've done a lot over the course of our project, starting with visualizing information from IMDb's website for the first season of <i>The Office</i>. 

From there, we worked on facial detection using OpenCV's LBPH classifier and evaluated the classifier's runtime against other facial detection models. Using the LBPH model for facial detection, we moved on into facial recognition using the LBPH technique of facial recognition. On our sample dataset of 4 characters, we managed an F1-score of 0.77. 

We improved the model's F1 score rating using image preprocessing and augmentation before exploring deep learning facial detection and classification techniques in the form of MTCNN and FaceNet, respectively. With these tools at our disposal, we created a high-performing facial recognition pipeline that can be used to classify faces from video data. 

With the ability to classify faces from videos, we measured character screen time by enumerating the number of frames each character's face was detected on the first season of <i>The Office</i>. We recorded these measurements on IMDb's website using BeautifulSoup to edit the website's HTML code.

---

Thank you all for reading this blog post and any blog posts that I've written so far. It's been an amazing learning experience diving into machine vision with this project and I hope you learned something about the modern day facial recognition technology that's so prevalent in society today.

Facial recognition is an amazing tool and very easy to implement in your own projects. Feel free to use the code and lessons I learned throughout this blog series to help you do cool stuff in your own personal projects. My name is Alden Chico, and it's been a pleasure going through this entire learning process with you all. I look forward to creating more projects in the near future. Until next time, have fun coding!